instalações necessárias


In [ ]:
import nltk 
nltk.download('all')
from nltk.corpus import floresta



definição e treinamento do tagger com o mac-morpho


In [2]:

def simplify_tag(t):
  if "+" in t:
    return t[t.index("+")+1:]
  else:
    return t

tsents = nltk.corpus.mac_morpho.tagged_sents()
tsents = [[(w.lower(),simplify_tag(t)) for (w,t) in sent] for sent in tsents if sent]
train = tsents
taggerMM = nltk.UnigramTagger(train)

funções

In [17]:
def RetiraStopWords(sentence):
  stopwords = ['.',',','hoje','e','muito','frequente','frequentes','esta','de', 'o']
  
  phrase = []
  for word in sentence:
    if word not in stopwords and word !='.':
      phrase.append(word)
  return phrase
 
def getApresentacao(palavra):  
  retorno = palavra
  
  retorno = retorno.replace('<com>','<possui>')
  retorno = retorno.replace('<apresenta>','<possui>')
  retorno = retorno.replace('<iniciou>','<realiza>')
  retorno = retorno.replace('<nega>','<**negação**>')
  retorno = retorno.replace('<não>','<**negação**>')
  
  
  return retorno.strip()  

def TransformacaoIntuitiva(frase):
  sentenca = frase.lower()
 
  sentenca = sentenca.replace('com diagnóstico de','diagnosticado')
  sentenca = sentenca.replace('encaminhada devido a','possui')
  sentenca = sentenca.replace('encaminhado devido a','possui')
  sentenca = sentenca.replace('devido a','possui')
  sentenca = sentenca.replace('recebeu diagnóstico de','possui')
  sentenca = sentenca.replace('em uso','usando')
  sentenca = sentenca.replace('refere','possui')
  sentenca = sentenca.replace('foram realizados','realizou')
  sentenca = sentenca.replace('foi avaliado','avaliado')
  sentenca = sentenca.replace('portador de','possui')
  sentenca = sentenca.replace('com quadro de','diagnosticado')
  sentenca = sentenca.replace('portadora de','diagnosticado')
  sentenca = sentenca.replace('vi','6')
  
  return sentenca

def Tokenizar(frase):
  return nltk.tokenize.word_tokenize((frase.lower().replace('.',' . ')+'.').replace('..','.'))

mostraLOG = False
def doLOG(frase):
  if mostraLOG:
    print(' ')
    print(frase)

def ExtrairSentencas(sentencas):   
  jaTeveAcao = True
  fraseFinal=[]
  palavrasDelimitadoras = ['com','não',',','.']
  sentenca=''
  for token in sentencas:
    if jaTeveAcao == False:
      jaTeveAcao = True
      continue;
    
    if token[0] in palavrasDelimitadoras or ('V' in str(token[1]) ):     
      if (sentenca!=''):
          fraseFinal.append(sentenca.strip())
          fraseFinal.append(token[0].strip())
          sentenca=''
          continue
      else:
        fraseFinal.append(token[0].strip())
    else:
      sentenca=sentenca+' '+token[0].strip()

  return RetiraStopWords(fraseFinal)

def TaggearMMorpho(tokens):
  lista=[]
  for token in tokens:
    lista.append(taggerMM.tag([token]))

  tokensRetorno=[]
  for token in lista:
    tokensRetorno.append(lista[0])

  return lista

def TransformacaoLimpeza(sentencasTaggeadas):
  sentencaFinal = []
  for palavra in sentencasTaggeadas:       
    valor = palavra[0][0]
    valorOK=''
    
    if (' em ' in valor):
      valor = valor.split(' em ')[0]
    if (' no ' in valor):
      valor = valor.split(' no ')[0]  
    if (' por ' in valor):
      valor = valor.split(' por ')[0]  
    
    sentencaFinal.append([valor,palavra[0][1]])
  return sentencaFinal

def ExtraiTriplas(sentencasLimpas):
  triplas=[]
  sentencaTripla=[]
  for token in sentencasLimpas:
    if ('ADV' in str(token[1]) and token[0] !='não'):
      continue
    
    if (('PREP' in str(token[1]) or 
         'V' in str(token[1]) or
         'PCP' in str(token[1]))):
      if len(sentencaTripla)>0: 
        triplas.append(sentencaTripla)
        sentencaTripla=[token]
      else:
        sentencaTripla=[token]
    else:
      sentencaTripla.append(token)
    
  triplas.append(sentencaTripla)
  return triplas

def ExtraiPredicado(triplas):
  triplaFinal=[]
  numLoop=0
  for tripla in triplas:
    numLoop=0
    predicado=''

    for sentenca in tripla:
      if numLoop==0:
        predicado=sentenca[0]
      numLoop+=1
      if numLoop>1:
        saida='<Paciente> ' +predicado+'<'+sentenca[0]+'> '
        triplaFinal.append([predicado,sentenca[0]])

  return triplaFinal

def LimparPredicados(predicados):
  triplaCorreta=[]
  for tripla in predicados:

    tokensPredicado = nltk.tokenize.word_tokenize(tripla[1])
    tagPredicado = taggerMM.tag(tokensPredicado)
    
    if ('PREP' in str(tagPredicado[0][1]) or 'PROADJ' in str(tagPredicado[0][1])):
      continue

    if (('era' in tripla[0]) or ('ocup'  in tripla[0])):
      continue
    
    triplaCorreta.append(tripla)
  return triplaCorreta

def Concatenar(lista):
  retorno=''
  for item in lista:
    retorno+=item+' '
  return retorno

def ExtrairRDF(predicado):
  numLoop=0
  alvo = Concatenar(RetiraStopWords(Tokenizar(predicado[1])))
  saida='<Paciente> ' +getApresentacao('<'+predicado[0]+'>')+ ' <'+getApresentacao(alvo)+'> '
        
  return saida

def TaggearTokens(tokens):
  tokensPreTaggeadas=[]
  for token in tokens:
    tokensPreTaggeadas.append(taggerMM.tag([token]))

  doLOG('TOKENS PRÉ TAGGEADAS:')
  doLOG(tokensPreTaggeadas)

  tokensTaggeadas = []
  for token in tokensPreTaggeadas:
    tokensTaggeadas.append(token[0])    
  return tokensTaggeadas


In [4]:

def analiseTeste(label,sentenca):
  doLOG('#######')
  doLOG('ORIGINAL:' + sentenca)  

  sentencaTransformada = TransformacaoIntuitiva(sentenca)

  tokens = Tokenizar(sentencaTransformada)
  
  tokensTaggeadas = TaggearTokens(tokens)  
  doLOG('TOKENS TAGGEADAS:')
  doLOG(tokensTaggeadas)

  sentencas = ExtrairSentencas(tokensTaggeadas)
  doLOG('SENTENCAS:')
  doLOG(sentencas)
    
  sentencasTaggeadas = TaggearMMorpho(sentencas)
  doLOG('SENTENCAS TAGGEADAS:')
  doLOG(sentencasTaggeadas)
  
  sentencasLimpas = TransformacaoLimpeza(sentencasTaggeadas)
  doLOG('SENTENCAS LIMPAS:')
  doLOG(sentencasLimpas)
  
  triplas = ExtraiTriplas(sentencasLimpas)
  doLOG('TRIPLAS:')
  doLOG(triplas)  
  
  triplasPredicado = ExtraiPredicado(triplas)
  doLOG('PREDICADOS:')
  doLOG(triplasPredicado)  

  predicadosLimpos = LimparPredicados(triplasPredicado)
  doLOG('PREDICADOS LIMPOS:')
  doLOG(predicadosLimpos)  
  
  print('')
  print('TRIPLAS historia #'+str(label))  
  for predicado in predicadosLimpos:
    fraseRDF = ExtrairRDF(predicado)    
    print(fraseRDF)
   



In [18]:
analiseTeste(1,'Paciente com HEMOFILIA A GRAVE em PROFILAXIA SECUNDÁRIA. Vem hoje com queixa de dor \
                em dorso do pé, nega traumas locais')
analiseTeste(2,'Paciente com Hemofilia A grave')
analiseTeste(3,'Paciente com Mucopolissacaridose tipo VI, Síndrome de Marateaux Lami iniciou terapia de \
                repozição enzimática em 30/04/2013')
analiseTeste(4,'Paciente com hemangioma em região toracica á direita, desde o nascimento. Essa lesão era \
                bem extensa ocupando 2/3 do hemitorax D , vem regredindo gradativamente')
analiseTeste(5,'Pciente portador de ANEMIA FALCIFORME,acompanha neste seviço desde o nascimento.Apresenta \
                crises dolorosas muito frequentes,internaçoes múltiplas,necessidade transfusional frequente. \
                Retorna para controle.Sofreu queimadura no ombro direito por bolsa de agua quente devido á dor,\
                com bolhas e sem infecção local.Esta com dor moderada no corpo todo')
analiseTeste(6,' Paciente com diagnóstico de meduloblastoma por anatomopatologico de cirurgia com ressecção \
                macroscópica incompleta em 08/08/12.      Não foi avaliado coluna e LCR negativo.')
analiseTeste(7,'É encaminhada devido a leucopenia (exame de origem leu=2.900). Queixa-se de dores em ambos os\
                 joelhos, sobretudo ao deambular. Sintomas agravaram-se após ganho de peso de quase 15Kg. Passou\
                 ontem por nutricionista e recebeu as orientações dietéticas cabíveis.Realizou acompanhamento na\
                  Hematologia até jan/2012, durante o qual recebeu diagnóstico de trombofilia.Hb=11,8  Ht=36,2%  \
                  Leu=3.040  Plaqu=182.000')
analiseTeste(8,'Vem encaminhada devido a plaquetopenia. Fev=100mil  Maio=60mil. Tem relato de equimoses espontâneas.')
analiseTeste(9,'Refere manchas no corpo todo, há 1 ano. Refere surgimento de feridas pruriginosas no corpo, \
                inicialmente em face, mãos e pernas, que progrediram para todo o corpo. Alega que fez tratamento \
                com ebastel e pomada de corticoide que nao recorda. Foram realizados exames para investigação de \
                alergias alimentares, sem alterações importantes, sugestivas. ')


TRIPLAS historia #1
<Paciente> <possui> <hemofilia a grave> 
<Paciente> <possui> <queixa dor> 
<Paciente> <**negação**> <traumas locais> 

TRIPLAS historia #2
<Paciente> <possui> <hemofilia a grave> 

TRIPLAS historia #3
<Paciente> <possui> <mucopolissacaridose tipo 6> 
<Paciente> <possui> <síndrome marateaux lami> 
<Paciente> <realiza> <terapia repozição enzimática> 

TRIPLAS historia #4
<Paciente> <possui> <hemangioma> 

TRIPLAS historia #5
<Paciente> <possui> <anemia falciforme> 
<Paciente> <acompanha> <neste se6ço desde nascimento> 
<Paciente> <possui> <crises dolorosas> 
<Paciente> <possui> <internaçoes múltiplas> 
<Paciente> <possui> <necessidade transfusional> 
<Paciente> <sofreu> <queimadura> 
<Paciente> <possui> <bolhas sem infecção local> 
<Paciente> <possui> <dor moderada> 

TRIPLAS historia #6
<Paciente> <possui> <ressecção macroscópica incompleta> 
<Paciente> <**negação**> <avaliado coluna lcr negativo> 

TRIPLAS historia #7
<Paciente> <possui> <leucopenia ( exame origem 

In [6]:
import functools

In [7]:
  compose = lambda *F: functools.reduce(lambda f, g: lambda x: f(g(x)), F)

  g = compose(TaggearTokens,
              Tokenizar,
              TransformacaoIntuitiva)
  print(g('Paciente possui hemofilia'))


[('paciente', 'N'), ('possui', 'V'), ('hemofilia', None), ('.', '.')]


In [185]:
def Stemar(frase):
  stemmer = nltk.stem.RSLPStemmer()
  tokens = nltk.tokenize.word_tokenize((frase.lower()))
  frasesStemadas = []
  palavraStemada=''
  palavraNormal=''
  for token in tokens:
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
  frasesStemadas.append((palavraStemada,palavraNormal))

  palavraStemada=''
  palavraNormal=''
  cont =1 
  for token in tokens:
    if cont<2:
      cont+=1
      continue;
      
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
    frasesStemadas.append((palavraStemada,palavraNormal))

  palavraStemada=''
  palavraNormal=''
  cont =1 
  for token in tokens:
    if cont<3:
      cont+=1
      continue;
      
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
    frasesStemadas.append((palavraStemada,palavraNormal))

  palavraStemada=''
  palavraNormal=''
  cont =1 
  for token in tokens:
    if cont<4:
      cont+=1
      continue;
      
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
    frasesStemadas.append((palavraStemada,palavraNormal))

  palavraStemada=''
  palavraNormal=''
  cont =1 
  for token in tokens[:-1]:
    if cont<2:
      cont+=1
      continue;
      
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
    frasesStemadas.append((palavraStemada,palavraNormal))

  palavraStemada=''
  palavraNormal=''
  cont =1 
  for token in tokens[:-1]:
    if cont<1:
      cont+=1
      continue;
      
    frasesStemadas.append((stemmer.stem(token),token))
    palavraStemada += stemmer.stem(token)
    palavraNormal += token+ ' '
    frasesStemadas.append((palavraStemada,palavraNormal))

  return frasesStemadas

def StemarUnitario(frase):
  stemmer = nltk.stem.RSLPStemmer()
  tokens = nltk.tokenize.word_tokenize((frase.lower()))
  frasesStemadas = []
  palavraStemada=''
  for token in tokens:
    palavraStemada += stemmer.stem(token)


  return palavraStemada

In [186]:

def simplify_tag(t):
  if "+" in t:
    return t[t.index("+")+1:]
  else:
    return t

tsents = nltk.corpus.mac_morpho.tagged_sents()

stemmer = nltk.stem.RSLPStemmer()

dataset = [[(StemarUnitario('dor de cabeça'),'sintoma'),
            (StemarUnitario('trombofilia'),'sintoma'),
            (StemarUnitario('necessidade transfusão'),'procedimento')
            ]]

print(dataset)
tsents = [[(w.lower(),simplify_tag(t)) for (w,t) in sent] for sent in dataset if sent]
train = tsents
taggerHC = nltk.UnigramTagger(train)

[[('dordecabeç', 'sintoma'), ('trombofil', 'sintoma'), ('necesstransfus', 'procedimento')]]


In [187]:
def ScanPredicado(frase):
  frasesStemadas = Stemar(frase.replace('<Paciente>','').replace('<','').replace('>',''))
 
  for frase in frasesStemadas:
    tokens = taggerHC.tag([frase[0]])
    #print(tokens)
    for token in tokens:
      #print(token[1])
      if (token[1] is not None):
        #print(token)
        return (token,frase[1].strip())
  #print('fin')
  return None

In [189]:
frases = []
frases.append('<paciente> <possui> <dor de cabeça>')
frases.append('<paciente> <tem> <dor de cabeça aguda>')
frases.append('<paciente> <relatou> <muitas dores de cabeça>')
frases.append('<paciente> <informou> <dores de cabeça matinais>')
frases.append('<paciente> <nega> <dores de cabeça>')

for frase in frases:
  print(ScanPredicado(frase))

#tokens = Tokenizar(frase)   
#tokens = RetiraStopWords(tokens)
#tokensRadical=[]
#for token in tokens:
#  tokensRadical.append(stemmer.stem(token))
#print(tokensRadical)



(('dordecabeç', 'sintoma'), 'dor de cabeça')
(('dordecabeç', 'sintoma'), 'dor de cabeça')
(('dordecabeç', 'sintoma'), 'dores de cabeça')
(('dordecabeç', 'sintoma'), 'dores de cabeça')
(('dordecabeç', 'sintoma'), 'dores de cabeça')


In [10]:
stemmer.stem("copiar")

'copi'

In [162]:
array = ['banana', 'maça', 'uva','pera']

In [163]:
array[:-1]

['banana', 'maça', 'uva']